# Linear elasticity: 2D cantilever under tip traction (plane strain)


In [50]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

 Geometry & mesh (rectangle with named sides)

In [51]:
shape = Rectangle(1.0, 1.0).Face()
shape.edges.Max(X).name = "right"
shape.edges.Min(X).name = "left"
shape.edges.Max(Y).name = "top"
shape.edges.Min(Y).name = "bottom"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.04))

Material (E, nu)

In [52]:
E, nu = 1.0, 0.20 
mu  = E/(2*(1+nu))
lam = E*nu/((1+nu)*(1-2*nu))    # 3D Lamé (plane strain default)

plane_stress = False
lam_eff = (2*mu*nu/(1-nu)) if plane_stress else lam


FE space (component-wise Dirichlet via a product space)

In [53]:
Vx = H1(mesh, order=2, dirichlet="left")     # only x-component fixed on 'left'
Vy = H1(mesh, order=2, dirichlet="bottom")   # only y-component fixed on 'bottom'
X  = Vx * Vy
(ux, uy), (vx, vy) = X.TnT()

Kinematics & constitutive law

In [54]:
def eps_2D(ux, uy):
    exx = grad(ux)[0]
    eyy = grad(uy)[1]
    exy = 0.5*(grad(ux)[1] + grad(uy)[0])
    return CoefficientFunction((exx, exy,   # row 0: [exx, exy]
                                exy, eyy),  # row 1: [exy, eyy]
                               dims=(2,2))

def sigma_2D(ux, uy):
    e  = eps_2D(ux, uy)
    tr = grad(ux)[0] + grad(uy)[1]           # exx + eyy
    return 2*mu*e + lam_eff*tr*Id(2)


Forms: a(u,v) and f(v)

In [55]:
a = BilinearForm(X, symmetric=True)
a += InnerProduct(sigma_2D(ux, uy), eps_2D(vx, vy)) * dx


Uniform upward traction on the right edge (adjust as you like)

In [56]:
t = CoefficientFunction((1.0, 0.0))
f = LinearForm(X)
f += InnerProduct(t, CoefficientFunction((vx, vy))) * ds("right")

Solve

In [57]:
with TaskManager():
    a.Assemble(); f.Assemble()
    gfu = GridFunction(X)
    gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * f.vec



 Visualize displacement & a stress component

In [58]:
uh = CoefficientFunction((gfu.components[0], gfu.components[1]))
Draw(Norm(uh), mesh, "||u||", deformation=uh)
Draw(sigma_2D(uh[0], uh[1])[0,0], mesh, "Sxx", deformation=uh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene